# "API Implementation"

- toc: true
- branch: master
- badges: true
- comments: true
- categories: [college board, java]

## Explanation

API's (Application Programming Interface) provide a way to store a large amount of data, and then allow you to access that data easily, without having to take it directly from another source. You can call API's in java and have that data be transferred. JSON is used to transmit this data. Since they're a massive data pool, you can easily take out data, allowing you to save time and code efficiently.

In [8]:
import java.net.URI;
import java.net.http.HttpClient;
import java.net.http.HttpRequest;
import java.net.http.HttpResponse;

HttpRequest request = HttpRequest.newBuilder()
		.uri(URI.create("https://wft-geo-db.p.rapidapi.com/v1/geo/cities/Q65/locatedIn"))
		.header("X-RapidAPI-Key", "f4dbb6f12cmsh8393e3c31992857p1076ddjsnf78a3b66c369")
		.header("X-RapidAPI-Host", "wft-geo-db.p.rapidapi.com")
		.method("GET", HttpRequest.BodyPublishers.noBody())
		.build();
HttpResponse<String> response = HttpClient.newHttpClient().send(request, HttpResponse.BodyHandlers.ofString());
System.out.println(response.body());

{"data":{"id":172136,"wikiDataId":"Q104994","type":"ADM2","city":"Los Angeles County","name":"Los Angeles County","country":"United States of America","countryCode":"US","region":"California","regionCode":"CA","elevationMeters":3,"latitude":34.05,"longitude":-118.25,"population":10014009,"timezone":"America__Los_Angeles","deleted":false}}


## Build a Web Service

This code provides output with information on certain cities. In this case, we are looking at Los Angeles County. We have access to information about this city, such as country, region, elevation, latitude, and population. 

In [16]:
import java.net.URI;
import java.net.http.HttpClient;
import java.net.http.HttpRequest;
import java.net.http.HttpResponse;

HttpRequest request = HttpRequest.newBuilder()
		.uri(URI.create("https://wft-geo-db.p.rapidapi.com/v1/geo/cities/Q65/locatedIn"))
		.header("X-RapidAPI-Key", "f4dbb6f12cmsh8393e3c31992857p1076ddjsnf78a3b66c369")
		.header("X-RapidAPI-Host", "wft-geo-db.p.rapidapi.com")
		.method("GET", HttpRequest.BodyPublishers.noBody())
		.build();
HttpResponse<String> response = HttpClient.newHttpClient().send(request, HttpResponse.BodyHandlers.ofString());
System.out.println(response.body());

import java.net.URI;
import java.net.http.HttpClient;
import java.net.http.HttpRequest;
import java.net.http.HttpResponse;
import java.util.Date;
import java.util.HashMap;

import org.json.simple.JSONObject;
import org.json.simple.parser.JSONParser;
import org.springframework.http.HttpStatus;
import org.springframework.http.ResponseEntity;
import org.springframework.web.bind.annotation.*;

@RestController // annotation to create a RESTful web services
@RequestMapping("/api/geodb-cities")  //prefix of API
public class CovidApiController {
    private JSONObject body; //last run result
    private HttpStatus status; //last run status
    String last_run = null; //last run day of month

    // GET Covid 19 Stats
    @GetMapping("/daily")   //added to end of prefix as endpoint
    public ResponseEntity<JSONObject> getGeo() {

        //calls API once a day, sets body and status properties
        String today = new Date().toString().substring(0,10); 
        if (last_run == null || !today.equals(last_run))
        {
            try {  //APIs can fail (ie Internet or Service down)

                //RapidAPI header
                HttpRequest request = HttpRequest.newBuilder()
		            .uri(URI.create("https://wft-geo-db.p.rapidapi.com/v1/geo/cities/Q65/locatedIn"))
		            .header("X-RapidAPI-Key", "f4dbb6f12cmsh8393e3c31992857p1076ddjsnf78a3b66c369")
		            .header("X-RapidAPI-Host", "wft-geo-db.p.rapidapi.com")
		            .method("GET", HttpRequest.BodyPublishers.noBody())
		            .build();

                //RapidAPI request and response
                HttpResponse<String> response = HttpClient.newHttpClient().send(request, HttpResponse.BodyHandlers.ofString());

                //JSONParser extracts text body and parses to JSONObject
                this.body = (JSONObject) new JSONParser().parse(response.body());
                this.status = HttpStatus.OK;  //200 success
                this.last_run = today;
            }
            catch (Exception e) {  //capture failure info
                HashMap<String, String> status = new HashMap<>();
                status.put("status", "RapidApi failure: " + e);

                //Setup object for error
                this.body = (JSONObject) status;
                this.status = HttpStatus.INTERNAL_SERVER_ERROR; //500 error
                this.last_run = null;
            }
        }

        //return JSONObject in RESTful style
        return new ResponseEntity<>(body, status);
    }
}

{"data":{"id":172136,"wikiDataId":"Q104994","type":"ADM2","city":"Los Angeles County","name":"Los Angeles County","country":"United States of America","countryCode":"US","region":"California","regionCode":"CA","elevationMeters":3,"latitude":34.05,"longitude":-118.25,"population":10014009,"timezone":"America__Los_Angeles","deleted":false}}


CompilationException: 